In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
# helper function to collect time consumption files
def collect_timefiles(root_path, verbose=0):
    result = []
    datasets = os.listdir(root_path)
    for i, d in enumerate(datasets):
        if verbose:
            print('='*40)
            print('Dataset:', d)
        expfolder = os.path.join(root_path, datasets[i])
        expfiles = [f for f in os.listdir(expfolder) if 'timebenchmark' in f]
        for j, s in enumerate(expfiles):
            if verbose:
                print('File:', s)
            result.append(os.path.join(expfolder, s))
    return result

# function to evaluate teh time consumption
def evaluate_timefiles(paths, return_print=True, return_plot=True, save=None):
    for p in paths:
        d = p.split('/')[-2]
        s = p.split('/')[-1].replace('timebenchmark_', '').replace('.pickle', '')
        print('='*40)
        print('Dataset:', d)
        print('Setup:', s)
        times = create_timedict(p)
        if not save is None:
            savefolder = os.path.join(save, d, s)
            if not os.path.exists(savefolder):
                os.makedirs(savefolder)
            print(savefolder)
        if return_print:
            print_times(times, save=savefolder if not save is None else save)
        if return_plot:
            plot_times(times, save=savefolder if not save is None else save)

# helper fuction to create time dict
def create_timedict(path):
    with open(path, "rb") as f:
        data = pickle.load(f)
    result = time_calculator(data)
    return result

# assigns correct tiems to methods
def time_calculator(timedict):
    comp_dict = {}
    # patchX
    if 'Training L2' in timedict:
        comp_dict['Training PatchX'] = timedict['Train Generator L1'] + timedict['Val Generator L1'] + timedict['Training L1'] + timedict['Train Prediction L1']
        comp_dict['Training PatchX'] += timedict['Val Prediction L1'] + timedict['Train Extraction L1'] + timedict['Val Extraction L1'] 
        comp_dict['Training PatchX'] += timedict['Train Extraction L2'] + timedict['Val Extraction L2'] + timedict['Training L2']
    if 'Test Prediction L2' in timedict:
        comp_dict['Inference PatchX'] = timedict['Test Generator L1'] + timedict['Test Prediction L1'] + timedict['Test Extraction L1'] 
        comp_dict['Inference PatchX'] += timedict['Test Extraction L2'] + timedict['Test Prediction L2']
    
    # patchX trivial
    if 'Training L1' in timedict:
        comp_dict['Training PatchX Trivial'] = timedict['Train Generator L1'] + timedict['Val Generator L1'] + timedict['Training L1'] 
    if 'Test Prediction Trivial' in timedict:
        comp_dict['Inference PatchX Trivial'] = timedict['Test Generator L1'] + timedict['Test Prediction L1'] + timedict['Test Extraction L1'] 
        comp_dict['Inference PatchX Trivial'] += timedict['Test Prediction Trivial']

    # blackbox
    if 'Training Blackbox' in timedict:
        comp_dict['Training Blackbox'] = timedict['Train Generator Blackbox'] + timedict['Val Generator Blackbox'] + timedict['Training Blackbox']
    if 'Test Prediction Blackbox' in timedict:
        comp_dict['Inference Blackbox'] = timedict['Test Generator Blackbox'] + timedict['Test Prediction Blackbox']

    # simple clf
    if 'Training SimpleClf_svm' in timedict:
        comp_dict['Training SimpleClf_svm'] = timedict['Training SimpleClf_svm']
    if 'Test Prediction SimpleClf_svm' in timedict:
        comp_dict['Inference SimpleClf_svm'] = timedict['Test Prediction SimpleClf_svm']

    # blackbox feature
    if 'Training BlackboxFeature' in timedict:
        comp_dict['Training BlackboxFeature'] = timedict['Train Featueextraction'] + timedict['Val Featueextraction'] + timedict['Train Generator BlackboxFeature']
        comp_dict['Training BlackboxFeature'] += timedict['Val Generator BlackboxFeature'] + timedict['Training BlackboxFeature']
    if 'Test Prediction BlackboxFeature' in timedict:
        comp_dict['Inference BlackboxFeature'] = timedict['Test Featueextraction'] + timedict['Test Generator BlackboxFeature']
        comp_dict['Inference BlackboxFeature'] += timedict['Test Prediction BlackboxFeature']

    # simple clf feature
    if 'Training SimpleClfFeature_svm' in timedict:
        comp_dict['Training SimpleClfFeature_svm'] = timedict['Train Featueextraction'] + timedict['Val Featueextraction']
        comp_dict['Training SimpleClfFeature_svm'] += timedict['Training SimpleClfFeature_svm']
    if 'Test Prediction SimpleClfFeature_svm' in timedict:
        comp_dict['Inference SimpleClfFeature_svm'] = timedict['Test Featueextraction'] + timedict['Test Prediction SimpleClfFeature_svm']

    return  comp_dict

# function to print the times
def print_times(time_dict, save=None):
    s = ''
    for i, k in enumerate(time_dict):
        if not i % 2:
            s += '='*40 + str('\n')
        s += 'Processing: ' + str(k) + str('\n')
        s += 'Seconds: ' + str(time_dict[k]) + str('\n')
    s = s[:-1]
    print(s)
        
    if not save is None:
        fname = 'TimeBenchmark.txt'
        with open(os.path.join(save, fname), "w") as f:
            f.write(s)

def plot_times(times, save=None):
    fig, axs = plt.subplots(1, 2, figsize=(10,4))
    #fig.suptitle('Time consumption benchmark')
    axes = axs.flat
    
    train_bars, infer_bars, approaches = [], [], []
    for i, k in enumerate(times):
        if  'Training' in k:
            train_bars.append(times[k])
            approaches.append(str(i//2) + ' ' + k.replace('Training ',''))
        else:
            infer_bars.append(times[k])
    n_app = len(approaches)
    colors = ['C' + str(i) for i in range(n_app)]

    axes[0].set_title('Training time')
    axes[1].set_title('Inference time')

    for i in range(n_app):
        axes[0].bar(i, train_bars[i], color=colors[i], label=approaches[i])
        axes[1].bar(i, infer_bars[i], color=colors[i], label=approaches[i])
        
    for ax in axes:
        ax.set(xlabel='Approach', ylabel='Time consumption [sec]')

    plt.legend(loc='upper left', bbox_to_anchor=(-1, -0.15),
          fancybox=True, shadow=True, ncol=3)


    if not save is None:
        fname = 'TimeBenchmark'
        plt.savefig(os.path.join(save, fname), dpi=300, bbox_inches='tight', pad_inches=0.1)
    
    plt.show()

In [ ]:
root_path = '../../models/'
stat_path = '../../statistics/'
paths = collect_timefiles(root_path)
evaluate_timefiles(paths, save=stat_path)